<a href="https://colab.research.google.com/github/XTWLP/RNN/blob/main/%E5%90%8D%E5%AD%97%E5%9B%BD%E5%AE%B6%E9%A2%84%E6%B5%8B_GRU%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import gzip
import csv
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.optim as optim
import math

class NameDataset(Dataset):
    def __init__(self, is_train_set=True):
        filename = 'names_train.csv.gz' if is_train_set else 'names_test.csv.gz'
        with gzip.open(filename, 'rt') as f:
            reader = csv.reader(f)
            rows = list(reader)
        self.names = [row[0] for row in rows]
        self.len = len(self.names)
        self.countries = [row[1] for row in rows]
        self.country_list = list(sorted(set(self.countries)))
        self.country_dict = self.getCountryDict()
        self.country_num = len(self.country_list)
        
    def __getitem__(self, index):
        return self.names[index], self.country_dict[self.countries[index]]
    
    def __len__(self):
        return self.len
    
    def getCountryDict(self):
        country_dict = dict()
        for idx, country_name in enumerate(self.country_list, 0):
            country_dict[country_name] = idx
        return country_dict
    
    def idx2country(self, index):
        return self.country_list[index]
    
    def getCountriesNum(self):
        return self.country_num

In [10]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
            super(RNNClassifier, self).__init__()
            self.hidden_size = hidden_size
            self.n_layers = n_layers
            self.n_directions = 2 if bidirectional else 1
            
            self.embedding = torch.nn.Embedding(input_size, hidden_size)
            self.gru = torch.nn.GRU(hidden_size, hidden_size, n_layers,
                                    bidirectional=bidirectional)
            self.fc = torch.nn.Linear(hidden_size * self.n_directions, output_size)
    
    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                            batch_size, self.hidden_size)
        return create_tensor(hidden)
    def forward(self, input, seq_lengths):
        # input shape : B x S -> S x B
        input = input.t()
        batch_size = input.size(1)
        
        hidden = self._init_hidden(batch_size)
        embedding = self.embedding(input)
        
        # pack them up
        gru_input = nn.utils.rnn.pack_padded_sequence(embedding, seq_lengths)
        
        output, hidden = self.gru(gru_input, hidden)
        if self.n_directions == 2:
            hidden_cat = torch.cat([hidden[-1], hidden[-2]], dim=1)
        else:
            hidden_cat = hidden[-1]
        fc_output = self.fc(hidden_cat)
        return fc_output

In [11]:
def name2list(name):
    arr = [ord(c) for c in name]
    return arr, len(arr)

def create_tensor(tensor):
    if USE_GPU:
        device = torch.device("cuda:0")
        tensor = tensor.to(device)
    return tensor

In [12]:
def make_tensors(names, countries):
        sequences_and_lengths = [name2list(name) for name in names]
        name_sequences = [sl[0] for sl in sequences_and_lengths]
        seq_lengths = torch.LongTensor([sl[1] for sl in sequences_and_lengths])
        countries = countries.long()

        # make tensor of name, BatchSize x SeqLen
        seq_tensor = torch.zeros(len(name_sequences), seq_lengths.max()).long()
        for idx, (seq, seq_len) in enumerate(zip(name_sequences, seq_lengths), 0):
            seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
       
        # sort by length to use pack_padded_sequence
        seq_lengths, perm_idx = seq_lengths.sort(dim=0, descending=True)
        seq_tensor = seq_tensor[perm_idx]
        countries = countries[perm_idx]
        
        return create_tensor(seq_tensor), \
                create_tensor(seq_lengths),\
                create_tensor(countries)

In [13]:
def trainModel():
    total_loss = 0
    for i, (names, countries) in enumerate(trainloader, 1):
        inputs, seq_lengths, target = make_tensors(names, countries)
        output = classifier(inputs, seq_lengths)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        if i % 10 == 0:
            print(f'[{time_since(start)}] Epoch {epoch} ', end='')
            print(f'[{i * len(inputs)}/{len(trainset)}] ', end='')
            print(f'loss={total_loss / (i * len(inputs))}')
    return total_loss

In [14]:
def testModel():
    correct = 0
    total = len(testset)
    print("evaluating trained model ...")
    with torch.no_grad():
        for i, (names, countries) in enumerate(testloader, 1):
            inputs, seq_lengths, target = make_tensors(names, countries)
            output = classifier(inputs, seq_lengths)
            pred = output.max(dim=1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
        percent = '%.2f' % (100 * correct / total)
        print(f'Test set: Accuracy {correct}/{total} {percent}%')
    
    return correct / total

In [15]:
# Parameters
HIDDEN_SIZE = 100
BATCH_SIZE = 256
N_LAYER = 2
N_EPOCHS = 100
N_CHARS = 128
USE_GPU = False

trainset = NameDataset(is_train_set=True)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testset = NameDataset(is_train_set=False)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

N_COUNTRY = trainset.getCountriesNum()

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
import time as time
if __name__ == '__main__':
    classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRY, N_LAYER)
    if USE_GPU:
        device = torch.device("cuda:0")
        classifier.to(device)
        
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
    
    start = time.time()
    print("Training for %d epochs..." % N_EPOCHS)
    acc_list = []
    for epoch in range(1, N_EPOCHS + 1):
        # Train cycle
        trainModel()
        acc = testModel()
        acc_list.append(acc)

Training for 100 epochs...
 1s] Epoch 1 [2560/13374] loss=0.008649419015273451
 3s] Epoch 1 [5120/13374] loss=0.007546639745123684
 4s] Epoch 1 [7680/13374] loss=0.0069094425377746425
 6s] Epoch 1 [10240/13374] loss=0.006482199358288199
 7s] Epoch 1 [12800/13374] loss=0.006098490729928016
evaluating trained model ...
Test set: Accuracy 4416/6700 65.91%
 10s] Epoch 2 [2560/13374] loss=0.00433320754673332
 11s] Epoch 2 [5120/13374] loss=0.0042062967433594165
 13s] Epoch 2 [7680/13374] loss=0.004057924216613174
 14s] Epoch 2 [10240/13374] loss=0.003971924143843353
 16s] Epoch 2 [12800/13374] loss=0.003899291269481182
evaluating trained model ...
Test set: Accuracy 4890/6700 72.99%
 18s] Epoch 3 [2560/13374] loss=0.003520940477028489
 20s] Epoch 3 [5120/13374] loss=0.003389047598466277
 21s] Epoch 3 [7680/13374] loss=0.003256725783770283
 23s] Epoch 3 [10240/13374] loss=0.0031741824408527465
 24s] Epoch 3 [12800/13374] loss=0.0030759233748540284
evaluating trained model ...
Test set: Accur